## [SF-DST] Car Price prediction Part2  
    Файлы входных данных доступны только для чтения в каталоге "../input/".
Например, при его запуске (щелкнув по кнопке «Выполнить» или нажав Shift + Enter) будут перечислены все файлы во входном каталоге.

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Вы можете записать до 20 ГБ в текущий каталог (/ kaggle / working /), который будет сохранен в виде вывода при создании версии с помощью «Сохранить и запустить все».
Вы также можете записывать временные файлы в / kaggle / temp /, но они не будут сохранены вне текущего сеанса.

In [1]:
# !pip install -q tensorflow==2.3

In [2]:
#аугментации изображений
!pip install albumentations -q

In [3]:
# В этой среде Python 3 установлено множество полезных аналитических библиотек.
# Он определяется в образе Docker kaggle/python: https://github.com/kaggle/docker-python
# Например, вот несколько полезных пакетов для загрузки

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import os
import sys
import PIL
import cv2
import re

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,RobustScaler
import seaborn as sns

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

import albumentations

# plt
import matplotlib.pyplot as plt

# увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

In [4]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [6]:
!pip freeze > requirements.txt

* ### Описание признаков в тестовом датасете:
* bodyType - тип кузова
* brand - марка автомобиля
* color - цвет автомобиля
* description - дополнительное описание автомобиля / объявления
* engineDisplacement - объем двигателя
* enginePower - мощность двигателя
* fuelType - тип топлива
* mileage - пробег
* modelDate - год выпуска модели
* model_info - информация о модели в оригинале и по-русски
* name - означение объема и типа двигателя, мощность в лошадиных силах
* numberOfDoors - кол-во дверей
* parsing_unixtime - дата парсинга данных
* price - цена
* productionDate - год производства
* sell_id - код (ID) продавца
* vehicleConfiguration -
* vehicleTransmission - тип коробки передачи
* Владельцы - кол-во владельцев
* Владение - срок владения
* ПТС - тип документа: оригинал или дупликат
* Привод - тип привода
* Руль - размещение руля

## DATA

Посмотрим на типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

In [7]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [8]:
# pd.set_option('display.max_columns', None)

In [9]:
train.info()
test.info()

In [10]:
# train.nunique()

## Model 1: Создадим "наивную" модель
Эта модель будет предсказывать среднюю цену по модели и году выпуска. C ней будем сравнивать другие модели.

In [11]:
# split данных
data_train, data_test = train_test_split(train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == {row[1]}"
    predicts.append(data_train.query(query)['price'].median())
    
# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

В принципе можно вместо median() взять mean()

# EDA

#### Для корректной обработки признаков объединяем трейн и тест в один датасет

In [12]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

In [13]:
data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

In [14]:
data.info()

### Проведем быстрый анализ данных для того, чтобы понимать характеристики признаков.

In [15]:
# используем модуль для быстрого первичного анализа данных
# import pandas_profiling
# pandas_profiling.ProfileReport(data)

### Проведем подробный анализ признаков.

#### Выделяем все числовые признаки и проведем первичную обработку

In [16]:
num_cols = data.select_dtypes(include=[np.number]).columns
# num_cols = data._get_numeric_data()
num_cols

In [17]:
"""
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
    columns = min(3, len(titles_values_dict))
    rows = (len(titles_values_dict) - 1) // columns + 1
    fig = plt.figure(figsize = (columns * 6, rows * 4))
    for i, (title, values) in enumerate(titles_values_dict.items()):
        hist, bins = np.histogram(values, bins = 20)
        ax = fig.add_subplot(rows, columns, i + 1)
        ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
        ax.set_title(title)
    plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna(),
})
"""

Эти числовые данные требуют логарифмирования, а затем нормироваия.

In [18]:
data['mileage'].isna().sum()

In [19]:
data['productionDate'].isna().sum()

In [20]:
"""
# Посмотрим на распределение целевой переменной:
plt.figure(figsize=(15, 3))
plt.subplot(1,2,1)
plt.title(f"Распределение {'price'}")
sns.distplot(data[data['sample'] == 1]['price'], bins=50)

plt.subplot(1,2,2)
sns.boxplot(data['price'])
plt.xlabel('Price')
plt.title(f"Боксплот {'price'}", fontsize=15)
plt.show()
# data.price.describe()
"""

In [21]:
"""
plt.figure(figsize=(15, 3))
plt.subplot(1,2,1)
plt.title(f"Распределение {'price'} после логарифмирования")
sns.distplot(np.log(data[data['sample'] == 1]['price']), bins=50)

plt.subplot(1,2,2)
sns.boxplot(np.log(data[data['sample'] == 1]['price']))
plt.xlabel('Price')
plt.title('Боксплот price после логарифмирования', fontsize=15)
plt.show()
"""

In [22]:
data[num_cols].corr()

Корректировка num_cols:

* sample - признак,характеризующий отношения данных к датасет train или test, удалить
* numberOfDoors - количество дверей авто, перенесем к категориальным признакам,удалить
* sell_id - код продавца, удалить
* price - целевой признак, удалять
* modelDate - сильно коррелируются с признаком productionDate,  удалять

Оставшие числовые данные требуют логарифмирования, а затем нормироваия.

В разделе PreProc Tabular Data введем новых признаков из числовых данных

### Выделяем категориальные признаки и проведем предварительной обработки

In [23]:
cat_cols = data.select_dtypes(include=[object]).columns
cat_cols

In [24]:
# data[cat_cols]

In [25]:
for col in cat_cols:
    print(col, '  ', data[col].isna().sum())

Признак "bodyType" имеет не однословную форму, необходимо привести к единой форме

In [26]:
data['bodyType'] = data['bodyType'].astype(str).apply(
    lambda x: None if x.strip() == '' else x)
data['bodyType'] = data.bodyType.apply(lambda x: x.split(' ')[0].lower())

data.bodyType.value_counts()

# data.bodyType.value_counts().plot.barh() 

In [27]:
data.brand.unique()

In [28]:
data.color.unique()

In [29]:
data.engineDisplacement.unique()

In [30]:
# извлечём цифры
data['engineDisplacement'] = data['engineDisplacement'].astype(str).apply(lambda x: x.split()[0])
# заменим неопределённость на число
data['engineDisplacement'] =  data['engineDisplacement'].replace('undefined', 0.01)
# приведём всё к одному формату
data['engineDisplacement'] = data['engineDisplacement'].astype(float)

In [31]:
data[['fuelType','name','vehicleConfiguration','engineDisplacement']][(
            data.engineDisplacement == 'undefined LTR')]

Эти  6 авто с не определенными литражами  являются авто с электро ('engineDisplacement'=='undefined LTR', "fuelType" == "электро")

In [32]:
data.enginePower.unique()

In [33]:
# Признак перевести в числовую. Предобработаем "enginePower"
data['enginePower'] = data['enginePower'].apply(lambda x: x[:3]).astype('int')

In [34]:
data.fuelType.unique()

In [35]:
data.fuelType.value_counts()

In [36]:
data.model_info.unique()

In [37]:
data.model_info.value_counts()

In [38]:
# data.name.unique()

В  признаке "name" имеющиеся информация есть в других признаках,значить надо удалять.
В BaseLine предлагает из признака "name" выделить значения 'xDrive'.

In [39]:
data['name_xDrive'] = data['name'].apply(lambda x: 1 if 'xDrive' in x else 0)

In [40]:
# data[data['name_xDrive']==1].value_counts().sum()

In [41]:
data['name_4wd_xDrive'] = data['name'].apply(lambda x: 1 if '4WD' in x else 0)

In [42]:
# data[data['name_4wd_xDrive']==1].value_counts().sum()

In [43]:
# data[(data['name_4wd_xDrive']==1) & (data['name_xDrive']==1)].sum()

In [44]:
data['name_4wd_xDrive'] = data['name'].apply(lambda x: 2 if 'xDrive' in x else 0)

In [45]:
# data[data['name_4wd_xDrive']==2].value_counts().sum()

In [46]:
data.drop('name_xDrive',  axis = 1, inplace=True)

Вместо прелагаемого признака 'name_xDrive' введен признак 'name_4wd_xDrive', которое присвоено значение: если признак 'name' содержит значения '4WD' и 'xDrive', то =2; если содержит только '4WD', то =1; в остальных случаях =0. (значение 'xDrive' без '4WD' в 'name'- нет ).

In [47]:
# data.info()

In [48]:
# data.vehicleConfiguration.unique()

В  признаке "vehicleConfiguration" имеющиеся информация есть в других признаках,значить надо удалять из датасет

In [49]:
data.vehicleTransmission.unique()

In [50]:
data.Владельцы.unique()

In [51]:
data.Владельцы.value_counts()

Признак "Владельцы" имеет единственный пропуск, по всей вероятности это "3 или более". Исправим и заменим на числовую.

In [52]:
data.Владельцы.fillna('3 или более', inplace = True)
data['Владельцы'] = data['Владельцы'].apply(lambda x: int(x[0])).astype('int32')

In [53]:
data.Владение.unique()

Признак "Владение" имеет слишком много пропусков, надо удалить.

In [54]:
data.ПТС.unique()

In [55]:
data.Привод.unique()

In [56]:
data.Руль.unique()

In [57]:
data.numberOfDoors.unique()

In [58]:
# корреляция между всех признаков
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax = sns.heatmap(data.corr(),fmt='.1g',
                 annot=True, cmap='coolwarm')

Сильная корреляция между "modelDate" и "productionDate". Удалить одного из них (возможно "modelDate")

### PreProc Tabular Data

In [59]:
data.info()

#### В категориальных признаках почти нет пропусков, кроме признака "Владение", один пропусок в признаке "Владельцы" легко исправимо. 
Признак "Владение" удалим из датасет, слишком много пропусков и из списка категориальных признаков.

Признак "bodyType" очищен и приведен к однословную форму.
 
Все текстовые признаки как категориальные за исключением признака "description", как длинный текстовой,  
 который требует NLP обработки. Добавляем к категориальным признак 'numberOfDoors' (число дверей авто), представленный в датасет как числовой признак
 
 В признаках "name" и  "vehicleConfiguration" имеющиеся информация есть в других признаках,значить их надо удалять из датасет
 (Но рекомендуется, что из признака "name"  выделить значения 'xDrive'). 
 (data['name_4wd_xDrive']  присвоили 2, если есть в строке 4WD и xDrive одновременно, если только 4WD, то 1, в остальных случаях 0. xDrive без 4WD в строоке name нет.

In [60]:
cat_cols = data.select_dtypes(include=[object]).columns
cat_cols

In [61]:
# Корректируем список категориальных признаков
col = ['description','name', 'vehicleConfiguration', 'Владение']
cat_cols = cat_cols.drop(col)

cat_cols = cat_cols.tolist()
cat_cols.append('Владельцы')
cat_cols.append('numberOfDoors')
cat_cols.append('name_4wd_xDrive')
cat_cols

In [62]:
num_cols = data.select_dtypes(include=[np.number]).columns
# num_cols = data._get_numeric_data()
num_cols

In [63]:
# Корректируем список числовых признаков
num_cols = num_cols.drop(['modelDate', 'Владельцы',  'numberOfDoors', 'sell_id', 'sample', 'price','name_4wd_xDrive',])
num_cols = num_cols.tolist()
num_cols.append('model_age')
num_cols.append('mile_per_year')
num_cols

Рекомендации, предложенные для улучшения Model 3 попробуем использовать и для модель 2:
* **df_output['model_age'] = np.log(2021 - df_output.modelDate)**
* **df_output['mile_per_year'] = np.log(df_output['mileage'] / df_output['model_age'])**
* Признак  **'engineDisplacement'** преобразовали в числовой.

Извлекли  числовых значений из текста: Парсинг признаков  **'enginePower'** для извлечения числовых значений.

Из признака 'name' выделили  значения **xDrive**  и **4WD** качестве отдельного признака, под именем **'name_4wd_xDrive'**.

In [64]:
def preproc_data(df_input):
    '''включает несколько функций предварительной обработки данных для прогноза '''
    df_output = df_input.copy()
        
    # ################### Feature Engineering ####################################################
    # тут ваш код на генерацию новых фитчей ...
    # сгенерируем новые признаки на основе существующих:
    
    # Новый признак - возраст модели:
    df_output['model_age'] = (2021 - df_output.modelDate) 
    
    # Новый признак - километраж за год. 
    df_output['mile_per_year'] = df_output['mileage']  /(2021 - df_output.modelDate)
    # Используем (2021 - df_output.modelDate) для случая отказа от новой фитчы 'model_age'
    
     # Логарифмы признаков
    df_output['model_age'] = np.log(df_output['model_age'])
    df_output['productionDate'] = np.log(data['productionDate'])
    # df_output['mile_per_year'] = np.log(df_output['mile_per_year'])
    # df_output['mileage'] = np.log(df_output['mileage'])
    
    # Удаление не нужных для модели признаки
    df_output.drop(['description','sell_id',  'name', 'vehicleConfiguration',
                    'modelDate', 'Владение'], axis = 1, inplace=True)
      
    # Нормализация данных
    # scaler = MinMaxScaler()
    scaler = RobustScaler() 
    for column in num_cols:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:, 0]

   ################### Категориальные признаки ########################################## 
    # Label Encoding
    for column in cat_cols:
        df_output[column] = df_output[column].astype('category').cat.codes
        
           
    # One-Hot Encoding: в pandas есть готовая функция - get_dummies.
    df_output = pd.get_dummies(
        df_output, columns=cat_cols, dummy_na=False)
     
    return df_output

In [65]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)
# df_preproc.head(5) # .sample(5)

In [66]:
df_preproc[num_cols].corr()

### Split data

In [67]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [68]:
# test_data.info()

## Model 2: CatBoostRegressor

In [69]:
model = CatBoostRegressor(iterations = 5000,
                          #depth=10,
                          #learning_rate = 0.01,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                         )

model.fit(X_train,np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=100,
         use_best_model=True,
        )

In [70]:
test_predict_catboost = np.exp(model.predict(X_test))
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

#### Submission

In [71]:
sub_predict_catboost = model.predict(X_sub)
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission.csv', index=False)

Рекомендации, предложенные для улучшения Model 3 model_age попробовали использовать и для модели 2 и 4:
Вновь введенный признак 'model_age' = 2021 - df_output.modelDate сильно коррелируется с признаком 'productionDate'. В Model 2: CatBoostRegressor эта зависимость дает устойчивый результат (11,21-11,22 %). 

Лучший результат (MAPE=11,15%) получается с тремя числовыми признаками:['enginePower', 'mileage', 'productionDate'] и со всеми  категориальными признаками.

## Model 3: Tabular NN
Построим обычную сеть:

### Simple Dense NN

In [72]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
# model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu")) # допонительный полносвязный слой
# model.add(L.Dropout(0.5))
# model.add(L.Dense(256, activation="relu"))
# model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))
# model.summary()

In [73]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [74]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor=['val_MAPE'], verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

#### Fit

In [75]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [76]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [77]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

In [78]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

Результаты экспериментов
* TEST mape:  10.53% ,  10.61%, 10.97%, 11.08%, 11.12%, 11.00%, 10.94%,  
* TEST mape:  10.68%, 10.50%,  10.66%, 10.60, 10.70%,10.58%, 10.44%, 10.51%, 10.59%
* TEST mape:  10.75%, 10.82%, 10.79%, 10.68%, 10.65%, 10.78%, 10.60%, 10.71%, 10.69%, 10.68%
* TEST mape: 10.53%, 10.55%, 

In [79]:
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:,0]
sample_submission.to_csv('nn1_submission.csv', index=False)

In [80]:
# sample_submission

### Рекомендации для улучшения Model 3:

* В нейросеть желательно подавать данные с распределением, близким к нормальному, поэтому от некоторых числовых признаков имеет смысл взять логарифм перед нормализацией. Пример: modelDateNorm = np.log(2020 - data['modelDate']) Статья по теме: https://habr.com/ru/company/ods/blog/325422

* Извлечение числовых значений из текста: Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.

* Cокращение размерности категориальных признаков Признак name 'name' содержит данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission'), поэтому эти данные можно удалить. Затем следует еще сильнее сократить размерность, например, выделив наличие xDrive в качестве отдельного признака.

### РезультатыModel 2 и Model 3.

Вновь введенные признаки 'model_age' (modelDateNorm) и 'mile_per_year' сильно коррелирются с признаками 'productionDate' и 'mileage'.

В Model 2: CatBoostRegressor эта зависимость дает устойчивый результат (Test_Mape = 11,21-11,22 %). Без введения новых фитчей результат - Test_Mape = 11,10% 

Model 3: Tabular NN работает не устойчиво (результат каждой запуски разные). Поэтому необходимо удалит одного из этих признаков, или 'model_age', или 'productionDate', или 'mileage', но их логарифмирование снижает корреляционной зависимости. Экспериментировано с этими признаками. Кроме того, проведен эксперимент с разной комбинацией логарифмирования числовых признаков

### Состав числовых признаков для всех моделей едины
Вновь введенные признаки 'model_age' (modelDateNorm) и 'mile_per_year' сильно коррелирются с признаками 'productionDate' и 'mileage'.

В Model 2: CatBoostRegressor эта зависимость дает устойчивый результат (Test_Mape = 11,21-11,22 %). Без введения новых фитчей результат - Test_Mape = 11,10% 

Model 3: Tabular NN работает не устойчиво (результат каждой запуски разные). Поэтому необходимо удалит одного из этих признаков, или 'model_age', или 'productionDate', или 'mileage', но их логарифмирование снижает корреляционной зависимости. Экспериментировано с этими признаками. Кроме того, проведен эксперимент с разной комбинацией логарифмирования числовых признаков

Вместо прелагаемого признака 'name_xDrive' введен признак 'name_4wd_xDrive', которое присвоено значение: если признак 'name' содержит значения '4WD' и 'xDrive', то =2; если содержит только '4WD', то =1; в остальных случаях =0. (значение 'xDrive' без '4WD' в 'name'- нет ).
Вместо прелагаемого признака 'name_xDrive' введен признак 'name_4wd_xDrive', которое присвоено значение: если признак 'name' содержит значения '4WD' и 'xDrive', то =2; если содержит только '4WD', то =1; в остальных случаях =0. (значение 'xDrive' без '4WD' в 'name'- нет ).
Вновь введенные признаки 'model_age' (modelDateNorm) и 'mile_per_year' сильно коррелирются с признаками 'productionDate' и 'mileage'.
В Model 2: CatBoostRegressor эта зависимость дает устойчивый результат (Test_Mape = 11,21-11,22 %). Без введения новых фитчей результат - Test_Mape = 11,10% 

Model 3: Tabular NN работает не устойчиво (результат каждой запуски разные). Поэтому необходимо удалит одного из этих признаков, или 'model_age', или 'productionDate', или 'mileage', но их логарифмирование снижает корреляционной зависимости. Экспериментировано с этими признаками. Кроме того, проведен эксперимент с разной комбинацией логарифмирования числовых признаков

Вместо прелагаемого признака 'name_xDrive' введен признак 'name_4wd_xDrive', которое присвоено значение: если признак 'name' содержит значения '4WD' и 'xDrive', то =2; если содержит только '4WD', то =1; в остальных случаях =0. (значение 'xDrive' без '4WD' в 'name'- нет ).

## Model 4: NLP + Multiple Inputs

In [81]:
!pip install pymorphy2
!pip install pymorphy2-dicts
# !pip install DAWG-Python
import nltk
import pymorphy2
nltk.download("stopwords")

In [82]:
df_preproc['description'] = data.description

In [83]:
morph = pymorphy2.MorphAnalyzer()
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

In [84]:
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        token = token.strip()
        token = morph.normal_forms(token)[0]
        tokens.append(token)
    return ' '.join(tokens)

In [85]:
df_preproc['description'] = df_preproc.apply(lambda df_preproc: lemmatize(df_preproc.description), axis=1)

In [86]:
df_preproc['description']

In [87]:
text = df_preproc['description'].max()
text

In [88]:
# Максимальное количество используемых слов. (наиболее часто)
MAX_WORDS = 100000
# Максимальное количество слов в каждой рекламации.
MAX_SEQUENCE_LENGTH = 256

In [89]:
# split данных
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]
text_sub = data.description.iloc[X_sub.index]

In [90]:
text_train

### Tokenizer

Здесь используем лемметизированные данные

In [91]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(df_preproc.description)

In [92]:
# tokenize.word_index

In [93]:
tokenize.document_count

In [94]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

In [95]:
# вот так теперь выглядит наш текст
print(text_train.iloc[6])

In [96]:
len(text_train.iloc[6])

In [97]:
print(text_train_sequences[6])

In [98]:
len(text_train_sequences[6])

### RNN NLP

Добавили еще один полносвязный слой с 64 нейронами и с activation="sigmoid" к модели  из BaseLine

In [99]:
# Модель для текстовой части (для признака 'description')
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
# model_nlp.add(L.BatchNormalization(axis=1)) #добавим BatchNormalization
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(64, activation="sigmoid")) 
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

### MLP

 Добавили еще один полносвязный слой с 256 нейронами к модели  из BaseLine

In [100]:
# Модель для табличной части
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
# model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
# model_mlp.add(L.Dropout(0.5))

### Multiple Inputs NN

Объединим модели nlp, mlp в Multi-Input сеть с помощью L.concatenate

In [101]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(128, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [102]:
# model.summary()

### Fit

In [103]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [104]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor='val_MAPE',
                             save_best_only=True, verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [105]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=256,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [106]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [107]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_mlp_nlp.hdf5')

In [108]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [109]:
sub_predict_nn2 = model.predict([text_sub_sequences, X_sub])
sample_submission['price'] = sub_predict_nn2[:,0]
sample_submission.to_csv('nn2_submission.csv', index=False)

Идеи для улучшения NLP части:

* Выделить из описаний часто встречающиеся блоки текста, заменив их на кодовые слова или удалив
* Сделать предобработку текста, например, сделать лемматизацию - алгоритм ставящий все слова в форму по умолчанию (глаголы в инфинитив и т. д.), чтобы токенайзер не преобразовывал разные формы слова в разные числа Статья по теме: https://habr.com/ru/company/Voximplant/blog/446738/
* Поработать над алгоритмами очистки и аугментации текста

### Model 5: Добавляем картинки

#### Data

In [110]:
# убедимся, что цены и фото подгрузились верно
plt.figure(figsize = (12,8))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [111]:
size = (320, 240)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

In [112]:
images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

In [113]:
X_train

### albumentations

In [114]:
from albumentations import (
    Compose, HorizontalFlip, Rotate, OneOf, MotionBlur, MedianBlur, Blur, 
    ShiftScaleRotate, OpticalDistortion, GridDistortion, PiecewiseAffine, 
    CenterCrop, RandomBrightnessContrast, 
    GaussianBlur, HueSaturationValue, RGBShift, FancyPCA, Resize
    )

In [115]:
albumentation = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=30, interpolation=1, border_mode=4,
                          value=None, mask_value=None, always_apply=False, p=0.5),
     OneOf([
        MotionBlur(p=0.2),
        MedianBlur(blur_limit=3, p=0.1),
        Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=1),
    OneOf([
        OpticalDistortion(p=0.3),
        GridDistortion(p=0.1),
        PiecewiseAffine (p=0.3),
    ], p=0.2),
    OneOf([
        CenterCrop(height=224, width=200),
        CenterCrop(height=200, width=224),
         RandomBrightnessContrast(
            brightness_limit=0.3, contrast_limit=0.3),
        RandomBrightnessContrast(
            brightness_limit=0.1, contrast_limit=0.1)
    ], p=0.5),
    GaussianBlur(p=0.05),
    HueSaturationValue(p=0.5),
    RGBShift(p=0.5),
    FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    Resize(240, 320)
])

In [116]:
#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = albumentation(image = images_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [117]:
def make_augmentations(images):
    print('применение аугментаций', end = '')
    augmented_images = np.empty(images.shape)
    for i in range(images.shape[0]):
        if i % 200 == 0:
            print('.', end = '')
        augment_dict = augmentation(image = images[i])
        augmented_image = augment_dict['image']
        augmented_images[i] = augmented_image
    print('')
    return augmented_images

### tf.data.Dataset
Если все изображения мы будем хранить в памяти, то может возникнуть проблема ее нехватки. Не храните все изображения в памяти целиком!

Метод **.fit()** модели keras может принимать либо данные в виде массивов или тензоров, либо разного рода итераторы, из которых наиболее современным и гибким является **tf.data.Dataset**. Он представляет собой конвейер, то есть мы указываем, откуда берем данные и какую цепочку преобразований с ними выполняем. Далее мы будем работать с **tf.data.Dataset**.

**Dataset** хранит информацию о конечном или бесконечном наборе **кортежей (tuple)** с данными и может возвращать эти наборы по очереди. Например, данными могут быть пары **(input, target)** для обучения нейросети. С данными можно осуществлять преобразования, которые осуществляются по мере необходимости (lazy evaluation).

**tf.data.Dataset.from_tensor_slices(data)** - создает датасет из данных, которые представляют собой либо массив, либо кортеж из массивов. Деление осуществляется по первому индексу каждого массива. Например, если **data = (np.zeros((128, 256, 256)), np.zeros(128))**, то датасет будет содержать **128** элементов, каждый из которых содержит один массив **256x256** и одно число.

**dataset2 = dataset1.map(func)** - применение функции к датасету; функция должна принимать столько аргументов, каков размер кортежа в **датасете 1** и возвращать столько, сколько нужно иметь в **датасете 2**. Пусть, например, датасет содержит изображения и метки, а нам нужно создать датасет только из изображений, тогда мы напишем так: 
**dataset2 = dataset.map(lambda img, label: img)**.

**dataset2 = dataset1.batch(8)** - группировка по батчам; если **датасет 2** должен вернуть один элемент, то он берет из **датасета 1** восемь элементов, склеивает их (нулевой индекс результата - номер элемента) и возвращает.

**dataset.__iter__()** - превращение датасета в итератор, из которого можно получать элементы методом **.__next__()**. Итератор, в отличие от самого датасета, хранит позицию текущего элемента. Можно также перебирать датасет циклом **for**.

* **dataset2 = dataset1.repeat(X)** - **датасет 2** будет повторять **датасет 1** **X** раз.

Если нам нужно взять из датасета **1000** элементов и использовать их как тестовые, а остальные как обучающие, то мы напишем так:

* **test_dataset = dataset.take(1000)**
* **train_dataset = dataset.skip(1000)**

Датасет по сути неизменен: такие операции, как **map, batch, repeat, take, skip**  никак не затрагивают оригинальный датасет. Если датасет хранит элементы **[1, 2, 3]**, то выполнив **3** раза подряд функцию **dataset.take(1)** мы получим 3 новых датасета, каждый из которых вернет число **1**. Если же мы выполним функцию **dataset.skip(1)**, мы получим датасет, возвращающий числа **[2, 3]**, но исходный датасет все равно будет возвращать **[1, 2, 3]** каждый раз, когда мы его перебираем.

**tf.Dataset** всегда выполняется в **graph**-режиме (в противоположность **eager**-режиму), поэтому либо преобразования (**.map()**) должны содержать только tensorflow-функции, либо мы должны использовать **tf.py_function** в качестве обертки для функций, вызываемых в **.map()**. Подробнее можно прочитать здесь.

In [118]:
# NLP part
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(df_preproc.description)

In [119]:
# tokenize.word_index

In [120]:
def process_image(image):
    return albumentation(image = image.numpy())['image']

def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price


In [121]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], y_test
    )).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)

In [122]:
'''
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices
    tf.config.experimental.set_memory_growth(device, True)
    
for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done
'''     

In [123]:
#проверяем, что нет ошибок (не будет выброшено исключение):
#train_dataset.__iter__().__next__();
#test_dataset.__iter__().__next__();
#sub_dataset.__iter__().__next__();

In [124]:
#train_dataset.__iter__().__next__();

In [125]:
#test_dataset.__iter__().__next__()

In [126]:
#sub_dataset.__iter__().__next__()

#### Строим сверточную сеть для анализа изображений без "головы"

In [127]:
# Нормализация включена в состав модели EfficientNetB3, 
# поэтому на вход она принимает данные типа uint8
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', 
                                    include_top = False, input_shape = (size[1], size[0], 3))

In [128]:
# В начале отметим обучаемыми все слои
efficientnet_model.trainable = True

# Вычислим количества половины слоев
fine_tune_at = len(efficientnet_model.layers)//2

# Все первые половины слоев оставим замороженными
for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [129]:
# проверим количество
print(len(efficientnet_model.layers))

# проверим статус слоев
# for layer in efficientnet_model.layers:
#    print(layer, layer.trainable)

In [130]:
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [131]:
# Модель для табличной части
tabular_model = Sequential([
    L.Input(shape = X_train.shape[1]),
    L.Dense(512, activation = 'sigmoid'),# изменили функцию activation
    L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    # L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    # L.Dropout(0.5)
    ])

In [132]:
# NLP Модель для текстовой части
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH),
    # L.LayerNormalization(), #добавили нормализацию
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.Dense(128, activation = 'sigmoid'), #добавили полносвязный слой с функцией активации sigmoid
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

In [133]:
# Функциональная модель
# Объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, 
                               tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, 
                      tabular_model.input, nlp_model.input], outputs=head)
# model.summary()

In [134]:
# объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, 
                               tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input,
                      tabular_model.input, nlp_model.input], outputs=head)
model.summary()

In [135]:
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [136]:
# Применим ExponentialDecay для управления learning rate
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate, decay_steps=100, decay_rate=0.9)

In [137]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor='val_MAPE',
                             save_best_only=True, verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]


In [138]:
history = model.fit(train_dataset.batch(30),
                    epochs=100,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [139]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [140]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [141]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [142]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission.csv', index=False)

#### Общие рекомендации:
* Попробовать разные архитектуры
* Провести более детальный анализ результатов
* Попробовать различные подходы в управление LR и оптимизаторы
* Поработать с таргетом
* Использовать Fine-tuning

#### Tabular
* В нейросеть желательно подавать данные с распределением, близким к нормальному, поэтому от некоторых числовых признаков имеет смысл взять логарифм перед нормализацией. Пример: modelDateNorm = np.log(2020 - data['modelDate']) Статья по теме: https://habr.com/ru/company/ods/blog/325422
 
* Извлечение числовых значений из текста: Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.
 
* Cокращение размерности категориальных признаков Признак name 'name' содержит данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission'). Можно удалить эти данные. Затем можно еще сильнее сократить размерность, например выделив наличие xDrive в качестве отдельного признака.
 
* Поработать над Feature engineering

#### NLP
* Выделить из описаний часто встречающиеся блоки текста, заменив их на кодовые слова или удалив
* Сделать предобработку текста, например сделать лемматизацию - алгоритм ставящий все слова в форму по умолчанию (глаголы в инфинитив и т. д.), чтобы токенайзер не преобразовывал разные формы слова в разные числа Статья по теме: https://habr.com/ru/company/Voximplant/blog/446738/
* Поработать над алгоритмами очистки и аугментации текста

#### CV
* Попробовать различные аугментации
* Fine-tuning

### Blend

In [143]:
blend_predict = (test_predict_catboost + test_predict_nn3[:,0]) / 2
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [144]:
blend_sub_predict = (sub_predict_catboost + sub_predict_nn3[:,0]) / 2
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)


## Выводы:¶
Выполнено очищение данных, их анализ, обработка и подготовка для дальнейшей работы. 

Над датасет проведена предварительная обработка, выполнена нормализация, использовались  методы кодирования категориальных признаков. 

Подготовлены дополнительные признаки. По итогам анализа, некоторые признаки удалены, (и на основе анализа корреляции). 

Проведены испытании с разными схемами подготовки числовых признаков.

Использовались классические методы обработки текста. В модели применялась токенизация и лемматизация. Результат Model 4: Test_MAPE = **10.71%**

Для использования в моделях изображении в основном использованы предложенные схемы в BaseLine и с использованием tf.data.Dataset. Однако, при объединении числовых, текстовых и изображении проверка с помощью предлагаемой инструкции (test_dataset.__iter__().__next__();
#sub_dataset.__iter__().__next__();) дает сбой, несмотря на уменьшение batch_size

Провели испытании с разными схемами аугментации на основе библиотеки albumentation.

Были проведены эксперименты с другими типовыми моделями EfficientNet, часть из которых приводила к переобучению. Были получены не очень хорошие результаты. TEST mape **10.99%**. Слишком большие зиг-заги метрики.

Придерживалась единая  архитектура предыдущей модели для последующих, т.е. модели для числовых таблиц  текстового признака единтичны для других моделей.

По итогам блендирования получились не хороших результатов метрики **10.52%**: . 

Что на данный момент соответствует 22 месту в лидерборде с метрикой модели: **10.93%**.

## Model Bonus: проброс признака

In [145]:
# MLP
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [146]:
# FEATURE Input
# Iput
productiondate = L.Input(shape=[1], name="productiondate")
# Embeddings layers
emb_productiondate = L.Embedding(len(X.productionDate.unique().tolist())+1, 20)(productiondate)
f_productiondate = L.Flatten()(emb_productiondate)

In [147]:
combinedInput = L.concatenate([model_mlp.output, f_productiondate,])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_mlp.input, productiondate], outputs=head)

model.summary()

In [148]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [149]:
history = model.fit([X_train, X_train.productionDate.values], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([X_test, X_test.productionDate.values], y_test),
                    callbacks=callbacks_list
                   )

In [152]:
'''
model.load_weights('../working/best_model.hdf5')
test_predict_nn_bonus = model.predict([X_test, X_test.productionDate.values])
print(f"TEST mape: {(mape(y_test, test_predict_nn_bonus[:,0]))*100:0.2f}%")
'''